## Prepare environment

In [12]:
import os
import random 
import csv 

from reportlab.pdfgen import canvas
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont

import email.message
import mimetypes
import smtplib

## Prepare variables

In [10]:
images = [os.path.join('img',img) for img in os.listdir('img')]
friends_source = open(os.path.join('data','friends.csv'), "r", encoding = 'utf-8')
csv_reader = csv.reader(friends_source)
friends = [tuple(friend) for friend in csv_reader][1:]
friends_source.close()

In [11]:
friends[0]

('Cesar', 'perezvaldesruben@gmail.com', '+525534600793')

In [3]:
image_settings = {'image1.webp':{'initial_width':115, 'font_color':(255, 232, 0)}, 
                  'image3.webp':{'initial_width':115, 'font_color':(255, 232, 0)}, 
                  'image2.webp':{'initial_width':80, 'font_color':(255, 232, 0)},
                  'image4.jpg':{'initial_width':115, 'font_color':(255, 255, 255)},
                  'image5.jpg':{'initial_width':10, 'font_color':(255, 255, 255)}}

## Prepare functions

In [4]:
def create_christmas_letter(friends, images, image_settings):
    """
    friends: list of tuples where the first element is a string representing a person's name.
    images: is a list of paths where the letter background images are stored.
    image_settings: is a dictionary having image name as key value and a nested key as corresponding value, for example:
     {'image1.webp':{'initial_width':115, 'font_color':(255, 232, 0)}}
    
    """
    for friend in friends:
        img = images[random.randint(0,len(images)-1)]
        img_name = img.split('\\')[-1]
        RGB_val = image_settings[img_name]['font_color']
        start_point_width = image_settings[img_name]['initial_width']
        start_point_height = 140
    
        title = f'Dear {friend[0]},'
        message = ['Merry Christmas!','','Thank you for the valuable gift','of your friendship.','','With love,','                César']
        filename = os.path.join('out',friend[0]+'.pdf')
    
        letter = canvas.Canvas(filename)
        pdfmetrics.registerFont(TTFont('Edwardian', 'ITCEDSCR.ttf'))
    
        letter.setPageSize((230, 180))
        letter.drawImage(img,0,0, 230, 180)
        letter.setFillColorRGB(RGB_val[0], RGB_val[1], RGB_val[2])
        letter.setFont('Edwardian', 20)
        letter.drawString(start_point_width, 160, title)
        
        height = start_point_height
        letter.setFont('Edwardian', 13)
        for phrase in message:
            letter.drawString(start_point_width, height, phrase)
            height = height - 15
        letter.save()
    return

In [6]:
def generate_email(friends, letter_path):
    """
    friends: list of tuples where the first element is a string representing a person's name, the second element is an email.
    letter_path: path to the folder where a set of pdf letters are being stored. 
    """
    for friend in friends:
        smtp = smtplib.SMTP_SSL("smtp.gmail.com")
        smtp.login('perezvaldesruben@gmail.com', "zxijjpzspchylslm")
        message = email.message.EmailMessage()
        message["From"] = 'perezvaldesruben@gmail.com'
        message["To"] = friend[1]
        message["Subject"] = 'Merry Christmas!'
        message.set_content(f'Hello {friend[0]}!!\n\nI wish you the merriest of christmas!')

        pdf_path = os.path.join(letter_path, friend[0]+'.pdf')
        mime_type, _ = mimetypes.guess_type(pdf_path)
        mime_type, mime_subtype = mime_type.split('/', 1)
    
        with open(pdf_path, 'rb') as ap:
            message.add_attachment(ap.read(),maintype=mime_type,subtype=mime_subtype,filename=friend[0]+'.pdf')
            
        smtp.send_message(message)
        smtp.quit()
    return 

## Test Run

In [5]:
create_christmas_letter(friends, images, image_settings)

In [7]:
generate_email(friends[0:1], 'out')